# Retrieve Odds
*Optional, but recommended for viewing model probabilities against bookmaker implied odds*

### Requirements:
- Api Key from [The Odds Api](https://the-odds-api.com/)

### Description: 
This notebook retrieves the current odds offered by bookmaker MyBookie. If you would like to change to a different sportsbook (there are many available from Odds-Api), it can be changed in "bookmakers" valuer in `request_params` in step (2).

### Instructions:
#### Initial set up:
1. Copy `.env.example` into a `.env` file at the project root
2. Obtain an api key from [The Odds Api](https://the-odds-api.com/)
3. Paste it into the `.env` file where directed.

#### Retrieve Odds
1. Enter the date of the desired match on line 14
2. Adjust bookmaker if desired on line 25.
**Odds Api bookmaker keys for convenience** - 
[US books](https://the-odds-api.com/sports-odds-data/bookmaker-apis.html#us-bookmakers) - 
[UK books](https://the-odds-api.com/sports-odds-data/bookmaker-apis.html#uk-bookmakers) - 
[EU books](https://the-odds-api.com/sports-odds-data/bookmaker-apis.html#eu-bookmakers)
3. Run all cells 
4. Confirm that data saved correctly to `data/processed/odds/odds_matchday_< matchday >.csv

*NOTE: Eventually this will be expanded to accomodate multiple sports books for comparison. For the moment, it only returns the odds for MyBookie where my humble bankroll currently lives*

In [1]:
import requests
from datetime import datetime
from dotenv import load_dotenv
import os
import json
import pandas as pd

load_dotenv()

odds_api_key = os.getenv("ODDS_API_API_KEY")
base_url = "https://api.the-odds-api.com/v4/sports/soccer_epl/odds"

"""!!! ENTER MATCHDAY HERE !!!"""
matchday = "2024-12-15"

request_params = {
    "apiKey": odds_api_key,
    "regions": "us",
    "markets": "h2h,totals",
    "oddsFormat": "decimal",
    "dateFormat": "iso",
    "commenceTimeFrom": f"{matchday}T00:00:00Z",
    "commenceTimeTo": f"{matchday}T23:59:59Z",
    # !!! ADJUST BOOKMAKER IF DESIRED !!!
    "bookmakers": "mybookieag"
}

response = requests.get(base_url, params=request_params)
if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}, {response.text}")


In [2]:
print(json.dumps(data, indent=2))

[
  {
    "id": "67f2a1bfcc3fe59328c79c3599490bfa",
    "sport_key": "soccer_epl",
    "sport_title": "EPL",
    "commence_time": "2024-12-15T16:30:00Z",
    "home_team": "Manchester City",
    "away_team": "Manchester United",
    "bookmakers": [
      {
        "key": "mybookieag",
        "title": "MyBookie.ag",
        "last_update": "2024-12-15T16:28:28Z",
        "markets": [
          {
            "key": "h2h",
            "last_update": "2024-12-15T16:28:28Z",
            "outcomes": [
              {
                "name": "Manchester City",
                "price": 1.54
              },
              {
                "name": "Manchester United",
                "price": 5.4
              },
              {
                "name": "Draw",
                "price": 4.7
              }
            ]
          },
          {
            "key": "totals",
            "last_update": "2024-12-15T16:28:28Z",
            "outcomes": [
              {
                "name": "Over",
 

In [3]:
odds_list = []

for match in data:
    home_team, away_team = match["home_team"], match["away_team"]
    matchup = f"{home_team} v. {away_team}"

    odds_data = {
        "match": matchup,
        "1": None,
        "x": None,
        "2": None,
        "ovr_und_point": None,
        "over": None,
        "under": None,
    }

    h2h_market = match["bookmakers"][0]["markets"][0]["outcomes"]
    totals_market = match["bookmakers"][0]["markets"][1]["outcomes"]

    for outcome in h2h_market:
        if outcome["name"] == home_team:
            odds_data["1"] = outcome["price"]
        elif outcome["name"] == away_team:
            odds_data["2"] = outcome["price"]
        elif outcome["name"] == "Draw":
            odds_data["x"] = outcome["price"]

    odds_data["ovr_und_point"] = totals_market[0]["point"]

    for outcome in totals_market:
        if outcome['name'] == "Over":
            odds_data["over"] = outcome["price"]
        elif outcome["name"] == "Under":
            odds_data["under"] = outcome["price"]


    odds_list.append(odds_data)

odds_df = pd.DataFrame(odds_list)
odds_df.to_csv(f"../data/processed/odds/odds_matchday_{matchday}.csv", index=False)
print(f"Data saved to '/data/processed/odds/odds_matchday_{matchday}.csv'")


Data saved to '/data/processed/odds/odds_matchday_2024-12-15.csv'
